### 操作說明：
* 方向鍵 ↑ ↓ ← → ：移動玩家的戰機
* 空白鍵：玩家的戰機發射子彈

### 遊戲機制
目前採取手動發射子彈的遊玩模式，每過5秒會補充子彈
* 分數計算的方式：
    1. 生存獎勵：每存活1秒獲得1分
    2. 擊殺獎勵：擊殺敵人獲得分數
* 擊殺敵人的方式：
    1. 用子彈擊殺：獲得正常的分數和經驗值
    2. 直接撞毀：獲得的分數和經驗值減半，而且玩家必須受到敵人攻擊力三倍的傷害
* 敵人會隨著時間經過越來越強 (目前敵人只有「隕石」，隨著時間經過移動速度會慢慢增加，出現頻率也會愈來愈高)
    
### 其他注意事項
* 程式還沒整理所以看起來很亂 > <

### 素材來源：
* 戰機: https://www.flaticon.com/free-icon/jet_2614542?term=jet&page=1&position=9
* 隕石: https://www.flaticon.com/free-icon/asteroid_2531034?term=meteorite&page=1&position=19
* 子彈：https://www.flaticon.com/free-icon/bullet_473433?term=bullet&page=1&position=3
* 爆炸特效：https://www.pngfuel.com/free-png/dsmti

In [ ]:
import pygame, sys
from pygame.locals import Color, QUIT, MOUSEBUTTONDOWN, USEREVENT, USEREVENT
from pygame.sprite import Group
from random import random, randint

score = 0
FPS = 60
player_width = 96
player_height = 96
bullet_width = 51
bullet_height = 17
bg_color = (0, 0, 0) # 背景顏色
screen_width = 1200
screen_height = 600

# 玩家操控的戰機的類別
class Player(pygame.sprite.Sprite):
    def __init__(self, width = player_width, height = player_height, level = 1, expRequired = 20, hpMax = 108, atk = 12):
        super().__init__()
        
        self.width = width
        self.height = height
        
        self.raw_image = pygame.image.load('./jet.png').convert_alpha()
        self.image = pygame.transform.scale(self.raw_image, (self.width, self.height))
        self.rect = self.image.get_rect()
        self.rect.topleft = (200, 300)
                             
        self.level = level # 玩家的等級
        self.expRequired = expRequired # 玩家升級所需的經驗值
        self.hpMax = hpMax # 玩家的生命上限
        self.atk = atk # 玩家的攻擊力
        self.hp = hpMax # 一開始的血量等於玩家的生命上限

        self.time = 0
        
        self.status = 'alive'
        
        self.bulletNumMax = 10 # 玩家目前的子彈數上限
        self.bulletNum = self.bulletNumMax # 玩家目前的子彈數
        
    def moveRight(self, pixels):
        self.rect.x += pixels
 
    def moveLeft(self, pixels):
        self.rect.x -= pixels

    def moveUp(self, pixels):
        self.rect.y -= pixels
        
    def moveDown(self, pixels):
        self.rect.y += pixels

    def update(self):
        # 經過一小段時間後還原圖片顏色(針對被擊中之後的閃爍事件)
        if pygame.time.get_ticks() - self.time >= 100: 
            self.image = pygame.transform.scale(self.raw_image, (self.width, self.height))
            
    def flash(self):
        self.image.fill((255, 0, 0), None, pygame.BLEND_RGBA_MULT)

class Bullet(pygame.sprite.Sprite):
    def __init__(self, init_x, init_y, width = 51, height = 17):
        super().__init__()
        
        self.width = width
        self.height = height
        
        self.raw_image = pygame.image.load('./bullet.png').convert_alpha()
        self.image = pygame.transform.scale(self.raw_image, (self.width, self.height))
        self.rect = self.image.get_rect()
        self.rect.x = init_x
        self.rect.y = init_y
        
        self.floating_point_x = init_x
        
    def update(self):
        ### 移動相關 ###
 
        # 實現每幀移動小數pixel的方法
        self.floating_point_x += 1.5
        self.rect.x = int(self.floating_point_x)
 
        # 超出邊界就消失
        if self.rect.x > screen_width:
            self.kill()


# 類別：隕石(敵人)
# AI：直線快速前進
class Enemy(pygame.sprite.Sprite):
    def __init__(self, imgDir, width = 128, height = 128, init_x = 1500, init_y = 300, hpMax = 50, 
                 atk = 15, speed = 0.5, score = 3):
        
        super().__init__()
        
        self.width = width
        self.height = width
        
        self.raw_image = pygame.image.load(imgDir).convert_alpha()
        self.image = pygame.transform.scale(self.raw_image, (width, height))
        self.rect = self.image.get_rect()
        self.rect.topleft = (init_x, randint(0 - int(self.height / 2), screen_height - int(self.height / 2))) 
            # y的範圍介於 視窗大小 + 半個自身高度 之間 (鼓勵玩家不要躲在邊邊)
        
        self.floating_point_x = init_x
        
        self.hpMax = hpMax # 玩家的生命上限
        self.atk = atk # 自己的攻擊力
        self.hp = hpMax # 一開始自己的血量等於自己的生命上限
        self.speed = speed # 自己的速度
        self.score = score # 自己被玩家擊殺時，玩家可以獲得的分數(撞毀的分數減半)
        
        self.time = 0
    
    def update(self):
        ### 移動相關 ###
 
        # 實現每幀移動小數pixel的方法
        self.floating_point_x -= self.speed 
        self.rect.x = int(self.floating_point_x)
 
        # 超出邊界就消失
        if self.rect.x < -self.width:
            self.kill()
            
        # 經過一小段時間後還原圖片顏色(針對被擊中之後的閃爍事件)
        if pygame.time.get_ticks() - self.time >= 100: 
            self.image = pygame.transform.scale(self.raw_image, (self.width, self.height))
    
    def flash(self):
        self.image.fill((255, 0, 0), None, pygame.BLEND_RGBA_MULT)

### 爆炸動畫 類別 ###
class Explosion(pygame.sprite.Sprite):
    def __init__(self, target, x, y):
        pygame.sprite.Sprite.__init__(self)
        self.target_surface = target
        self.image = None
        self.master_image = None
        self.rect = None
        self.topleft = 0,0
        self.frame = 0
        self.old_frame = -1
        self.frame_width = 1
        self.frame_height = 1
        self.first_frame = 0
        self.last_frame = 0
        self.columns = 1
        self.last_time = 0
        self.master_image = pygame.image.load('./explosion.png').convert_alpha()
        self.frame_width = 192
        self.frame_height = 192
        self.rect = x,y,192,192
        self.columns = 5
        rect = self.master_image.get_rect()
        self.last_frame = (rect.width // 192) * (rect.height // 192) - 1

    def update(self, current_time, rate=30):
        if current_time > self.last_time + rate:
            self.frame += 1
            if self.frame > self.last_frame:
                self.frame = self.first_frame
                self.kill()
            self.last_time = current_time

        if self.frame != self.old_frame:
            frame_x = (self.frame % self.columns) * self.frame_width
            frame_y = (self.frame // self.columns) * self.frame_height
            rect = ( frame_x, frame_y, self.frame_width, self.frame_height )
            self.image = self.master_image.subsurface(rect)
            self.old_frame = self.frame


### 創造文字物件 ###
#參數：視窗畫布、文本、字型、字型大小、字框底色、(字框x座標、字框y座標  字框寬度、字框高度)        
class textBox():
    def __init__(self, screen, text, x, y, width, height, font_color = (0,0,0), font_family = 'Calibri', 
            font_size = 36, box_color = (128,128,128)):
        self.x, self.y, self.width, self.height = x, y, width, height
        pygame.font.init()
        head_font = pygame.font.SysFont(font_family, font_size)
        text_surface = head_font.render(text, True, font_color)
        text_rect = text_surface.get_rect(center=(x, y))
        width_text, height_text = head_font.size(text)
        self.area = pygame.draw.rect(screen, box_color, (x - width / 2, y - height / 2, width, height), 0)
        screen.blit(text_surface, text_rect)


def main():
    # 初始化
    pygame.init()
    
    # 設置timer
    clock = pygame.time.Clock()
    clock2 = pygame.time.Clock()
    
    # 建立 window 視窗畫布
    window_surface = pygame.display.set_mode((screen_width, screen_height))
    # 設置視窗標題為
    pygame.display.set_caption('Rookie Fighter (ver: 20200313)')
    # 宣告 font 文字物件
    head_font = pygame.font.SysFont('Calibri', 30)
    # 更新畫面，等所有操作完成後一次更新（若沒更新，則元素不會出現）
    pygame.display.update()
    
    score = 0
    FPS = 60
    
    screen=pygame.display.get_surface()
    
    # 創建玩家
    player = Player()
    
    # 創建存放子彈的群組
    bullets = pygame.sprite.Group()

    # 創建存放敵人的群組
    enemys = pygame.sprite.Group()
    speed_new = 0.5

    # 創建存放爆炸動畫的群組
    explosions = pygame.sprite.Group()
    
    time_counter = 0
    time_counter2 = 0
    
    lower_limit = 1000
    upper_limit = 2000
    time_threshold = randint(lower_limit, upper_limit) # 1~2秒之後出現第一個敵人
    
    game_state = 'main_menu'
    
    ticks_death = 0
    ticks_addBullet = 0
    
    # 事件迴圈監聽事件，進行事件處理
    while True:
        
        ### 遊戲主界面 ###
        if game_state == 'main_menu':

            # 玩家成績儀表版
            head_font2 = pygame.font.SysFont('Calibri', 72)
            text_surface2 = head_font2.render('ROOKIE FIGHTER', True, (255, 255, 255))
            text_rect2 = text_surface2.get_rect(center=(screen_width/2, screen_height/2 * 0.4))
            window_surface.blit(text_surface2, text_rect2)
            
            text_surface = head_font.render('ver 2020/3/13', True, (255, 255, 255))
            text_rect = text_surface.get_rect(center=(screen_width/2, screen_height/2 * 0.6))
            window_surface.blit(text_surface, text_rect)
            
            start_button = textBox(screen = screen, text = 'START', 
                                          x = screen_width / 2, y = screen_height / 2 * 1, width = 150, height = 70, 
                                          font_color = (255,255,255))
            quit_button = textBox(screen = screen, text = 'QUIT', 
                                          x = screen_width / 2, y = screen_height / 2 * 1.4, width = 150, height = 70, 
                                          font_color = (255,255,255))
            
            # 迭代整個事件迴圈，若有符合事件則對應處理
            for event in pygame.event.get():
                # 當使用者結束視窗，程式也結束
                if event.type == QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.MOUSEBUTTONDOWN:
                    if event.button == 1:  # Left mouse button.
                        # Check if the rect collides with the mouse pos.
                        if start_button.area.collidepoint(event.pos):
                            game_state = 'playing'
                        if quit_button.area.collidepoint(event.pos):
                            pygame.quit()
                            sys.exit()

            pygame.display.update()
            clock.tick(15)
            
        
        elif game_state == 'playing':

            # 玩家死亡
            if player.hp <= 0 and player.status == 'alive':
                player.hp = 0
                explosions.add(Explosion(screen, 
                         player.rect.x + (player.width - 192) / 2, 
                         player.rect.y + (player.height - 192) / 2))
                player.rect.x = -1000
                player.rect.y = -1000
                player.status = 'death'
                ticks_death = pygame.time.get_ticks()
                
            if pygame.time.get_ticks() - ticks_death >= 3000 and player.status == 'death':
                game_state = 'game_over'

            # 玩家和敵人的碰撞偵測
            for enemy in enemys:
                if pygame.sprite.collide_rect(player, enemy):
                    enemy.hp = 0
                    explosions.add(Explosion(screen, 
                                             enemy.rect.x + (enemy.width - 192) / 2, 
                                             enemy.rect.y + (enemy.height - 192) / 2))
                    player.time = pygame.time.get_ticks()
                    player.flash()
                    player.hp -= (enemy.atk * 3)
                    score += int(enemy.score / 2)
                    enemy.kill()
                    
                    
            # 玩家子彈和敵人的碰撞偵測
            for bullet in bullets:
                for enemy in enemys:
                    if pygame.sprite.collide_rect(bullet, enemy):
                        enemy.time = pygame.time.get_ticks()
                        enemy.flash()
                        bullet.kill()
                        enemy.hp -= player.atk
                        if enemy.hp <= 0:
                            explosions.add(Explosion(screen, 
                                                     enemy.rect.x + (enemy.width - 192) / 2, 
                                                     enemy.rect.y + (enemy.height - 192) / 2))
                            enemy.kill()
                            score += enemy.score
                            
            # 每過一段時間出現新的敵人
            time_counter += clock.tick()
            if time_counter >= time_threshold:
                speed_new += random() * 0.025 # 新的敵人將擁有更快的速度(每畫格增加0~0.025)
                width_new = height_new = randint(64,160) # 新的敵人會有不同的大小
                enemys.add(Enemy(imgDir = '.\meteorite.png', width = width_new, height = height_new, 
                                 speed = speed_new, score = 10)) 
                time_counter = 0
                
                lower_limit -= 25
                if lower_limit <= 250:
                    lower_limit = 250
                    
                upper_limit -= 25
                if upper_limit <= 500:
                    upper_limit = 500

                time_threshold = randint(lower_limit, upper_limit) # 一段時間後出現下一個敵人(之後出現頻率會愈來愈高)
            
            # 持續存活分數獎勵
            time_counter2 += clock2.tick()
            
            if time_counter2 >= 1000 and player.status == 'alive':
                score += 1 # 每存活1秒加1分
                time_counter2 = 0
            
            # 每隔5秒玩家的戰機將自動補充子彈
            if pygame.time.get_ticks() - ticks_addBullet >= 5000 and player.status == 'alive':
                player.bulletNum = player.bulletNumMax
                ticks_addBullet = pygame.time.get_ticks()

            # 背景著色
            window_surface.fill(bg_color)

            # 玩家戰機的四個方位移動(配合按鍵)
            keys_pressed = pygame.key.get_pressed()

            if keys_pressed[pygame.K_LEFT]:
                if player.rect.x >= 0:
                    player.moveLeft(1)

            if keys_pressed[pygame.K_RIGHT]:
                if player.rect.x <= (screen_width - player.width):
                    player.moveRight(1)

            if keys_pressed[pygame.K_UP]:
                if player.rect.y >= 0:
                    player.moveUp(1)

            if keys_pressed[pygame.K_DOWN]:
                if player.rect.y <= (screen_height - player.height):
                    player.moveDown(1)
            
            # 迭代整個事件迴圈，若有符合事件則對應處理
            for event in pygame.event.get():
                # 當使用者結束視窗，程式也結束
                if event.type == QUIT:
                    pygame.quit()
                    sys.exit()
                # 按鍵的按下偵測
                if event.type == pygame.KEYDOWN:
                    # 按下空白鍵，玩家的戰機將會發射子彈
                    if event.key == pygame.K_SPACE and player.bulletNum > 0:
                        bullets.add(Bullet(init_x = player.rect.x + (player.width - bullet_width) / 2, 
                                           init_y = player.rect.y + (player.height - bullet_height) / 2))
                        player.bulletNum -= 1
            
            # 玩家的戰機更新
            player.update()
            
            # 子彈畫面的更新
            bullets.draw(screen)
            bullets.update()                    
                    
            # 敵人畫面的更新
            enemys.draw(screen)
            enemys.update()
            
            # 特效畫面的更新
            ticks = pygame.time.get_ticks()
            explosions.update(ticks)
            explosions.draw(screen)

            # 渲染物件
            if player.hp > 0:
                window_surface.blit(player.image, player.rect)

            # 玩家資訊儀表(顯示等級、生命值、分數...等資訊)
            text_surface = head_font.render('HP: {} / {}'.format(player.hp, player.hpMax), True, (255, 255, 255))
            window_surface.blit(text_surface, (10, 10))
            text_surface = head_font.render(
                'Bullet: {} / {}'.format(player.bulletNum, player.bulletNumMax), True, (255, 255, 255))
            window_surface.blit(text_surface, (10, 40))
            text_surface2 = head_font.render('Score: {}'.format(score), True, (255, 255, 255))
            window_surface.blit(text_surface2, (10, 70))

            pygame.display.update()
        
        elif game_state == 'game_over':
            
            # 迭代整個事件迴圈，若有符合事件則對應處理
            for event in pygame.event.get():
                # 當使用者結束視窗，程式也結束
                if event.type == QUIT:
                    pygame.quit()
                    sys.exit()
                    
            window_surface.fill(bg_color)
            
            # 玩家成績儀表版
            head_font2 = pygame.font.SysFont('Calibri', 72)
            text_surface2 = head_font2.render('GAME OVER', True, (255, 255, 255))
            text_rect2 = text_surface2.get_rect(center=(screen_width/2, screen_height/2))
            window_surface.blit(text_surface2, text_rect2)
            
            text_surface = head_font.render('Yor score: {}'.format(score), True, (255, 255, 255))
            text_rect = text_surface.get_rect(center=(screen_width/2, screen_height/2 * 1.2))
            window_surface.blit(text_surface, text_rect)

            pygame.display.update()


if __name__ == '__main__':
    main()
        